In [2]:
from llm_dev.ch2.simple_tokenizer import SimpleTokenizerV1
from llm_dev.ch2.read_text import read_text_file
from llm_dev.ch2 import pre_process_text

In [3]:
input_file = "../../../the-verdict.txt"

In [4]:
full_text = read_text_file(input_file)
vocab = pre_process_text.create_vocab(full_text)
tokenizer = SimpleTokenizerV1(vocab)

Number of tokens: 4690
The generated vocabulary size: 1130


In [5]:
text = "Hello, do you like tea?"
print(tokenizer.encode(text))

Number of tokens: 7


KeyError: 'Hello'

### 2.4 Adding special context tokens

In [6]:
vocab_v2 = pre_process_text.create_vocab_v2(full_text)

Number of tokens: 4690
The generated vocabulary size: 1132


In [7]:
for i, item in enumerate(list(vocab_v2.items())[-5:]):
    print(i, item)

0 ('younger', 1127)
1 ('your', 1128)
2 ('yourself', 1129)
3 ('<|unk|>', 1130)
4 ('<|endoftext|>', 1131)


### 2.5 Byte pair encoding

In [8]:
from importlib.metadata import version
import tiktoken
print("tiktoken version:", version('tiktoken'))

tiktoken version: 0.9.0


In [9]:
tokenizer = tiktoken.get_encoding("gpt2")

In [10]:
text = "Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPalace."
integers = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
print(integers)
print(tokenizer.decode(integers))

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 286, 617, 34680, 11531, 558, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terraces of someunknownPalace.


#### Excercise 2.1 Byte Pair encoding of unknown words

In [11]:
ids = tokenizer.encode("Akwirw ier")

print(ids)

In [12]:
for single_id in ids:
    print(tokenizer.decode([single_id]))

Ak
w
ir
w
 
ier


In [13]:
tokenizer.decode(ids)

'Akwirw ier'

### Section 2.6: Data Sampling

In [14]:
text = read_text_file(input_file)
enco_text = tokenizer.encode(text)

In [15]:
enc_sample = enco_text[:50]

In [16]:
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

In [17]:
print(f"x: {x}")
print(f"y:     {y}")

x: [40, 367, 2885, 1464]
y:     [367, 2885, 1464, 1807]


#### Exercise 2.2: Use data loader with the dataset

In [18]:
with open(input_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

In [19]:
from llm_dev.ch2.data_loader import create_dataloader_v1


data_loader = create_dataloader_v1(
    txt=raw_text,
    batch_size=3,
    max_length=4,
    stride=4,
    shuffle=False,
)
data_iter = iter(data_loader)

In [20]:
batch = next(data_iter)


In [21]:
print(batch)

[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026]]), tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632]])]


In [22]:
len(batch)

2

In [23]:
batch_2 = next(data_iter)
print(batch_2)

[tensor([[15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645]]), tensor([[ 438, 2016,  257,  922],
        [5891, 1576,  438,  568],
        [ 340,  373,  645, 1049]])]


### Section 2.7: Text Embedding
A hands-on example on how embedding layer works.

In [24]:
import torch
vocab_size = 6
output_dim = 3

torch.manual_seed(42)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)
print(embedding_layer.weight)

Parameter containing:
tensor([[ 1.9269,  1.4873, -0.4974],
        [ 0.4396, -0.7581,  1.0783],
        [ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [-0.2316,  0.0418, -0.2516],
        [ 0.8599, -0.3097, -0.3957]], requires_grad=True)


In [25]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.6866,  0.6105,  1.3347]], grad_fn=<EmbeddingBackward0>)


In [26]:
input_ids = torch.tensor([2, 3, 5, 1])
print(embedding_layer(input_ids))

tensor([[ 0.8008,  1.6806,  0.3559],
        [-0.6866,  0.6105,  1.3347],
        [ 0.8599, -0.3097, -0.3957],
        [ 0.4396, -0.7581,  1.0783]], grad_fn=<EmbeddingBackward0>)


### Section 2.8: Position Encoding

In [27]:
vocab_size = 50257
output_dim = 256
token_embedding_layer = torch.nn.Embedding(vocab_size, output_dim)


In [28]:
max_length = 4
dataloader = create_dataloader_v1(
    txt=raw_text, batch_size=8, max_length=max_length, stride=max_length, shuffle=False
)

In [29]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("Token IDs: \n", inputs)
print("Inputs shape:\n", inputs.shape)

Token IDs: 
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Inputs shape:
 torch.Size([8, 4])


In [30]:
print(targets)
print(targets.shape)

tensor([[  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632],
        [  438,  2016,   257,   922],
        [ 5891,  1576,   438,   568],
        [  340,   373,   645,  1049],
        [ 5975,   284,   502,   284],
        [ 3285,   326,    11,   287]])
torch.Size([8, 4])


In [31]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [32]:
from turtle import pos


context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length, output_dim)
pos_embeddings = pos_embedding_layer(torch.arange(context_length))
print(pos_embeddings.shape)

torch.Size([4, 256])


In [33]:
pos_embeddings.shape

torch.Size([4, 256])

tensor([ 1.2192, -0.2741,  0.6823, -0.1669, -0.8921,  0.1055, -0.3790,  0.5566,
        -0.6532,  0.6945,  0.2294,  0.7706, -1.0600,  1.1109, -0.0416,  0.0755,
        -0.4250, -0.6974,  1.9947, -0.1386,  0.6699, -0.1933, -0.6626, -2.5877,
         0.1789,  1.3156,  1.2939, -0.6690, -0.7701, -0.2048,  2.0718, -1.4005,
        -0.1533, -0.9274, -1.6961,  0.7975,  0.1333, -0.3854,  1.7930, -0.5936,
         1.0393, -0.2826, -0.7999, -0.3349,  0.3119,  1.9270,  0.6508,  1.3521,
        -0.5486, -0.0687, -0.5394,  1.2094,  1.2658, -0.6468,  0.9397, -0.2972,
         1.5191, -0.9091,  0.3122, -0.8711,  1.0148, -0.5497,  2.1036, -0.6052,
         0.6682,  0.8231,  1.1816,  1.1979,  0.8280, -1.4040,  1.0860,  1.1826,
         0.5376, -0.5385, -0.0396,  0.2138, -0.1422,  0.9105, -0.0212, -1.2918,
         0.4179,  0.8971, -0.9209, -0.1980,  0.3185,  0.7440, -0.8477, -0.8741,
         1.1092, -0.0516,  0.5348, -0.4087,  1.0480,  1.3445,  0.8101,  0.9920,
         1.0377, -0.2992, -0.8859,  0.19

In [34]:
input_embeddings = token_embeddings + pos_embeddings

In [35]:
input_embeddings.shape

torch.Size([8, 4, 256])